In [ ]:
# 🚀 Install required packages for running the model
# - transformers: for loading and running LLMs
# - datasets: optional, for dataset handling
# - peft: for later LoRA experiments
# - accelerate: for optimized GPU usage in Colab
!pip install -q transformers datasets peft accelerate

In [ ]:
# 🔍 Check GPU availability in Colab
import torch
if torch.cuda.is_available():
    print("GPU is available:", torch.cuda.get_device_name(0))
else:
    print("GPU not available. Make sure Runtime -> Change runtime type -> GPU is selected")

In [ ]:
# 🔍 Check GPU properties and memory in Colab
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f"✅ GPU is available: {torch.cuda.get_device_name(device)}")
    print(f"  - CUDA version: {torch.version.cuda}")
    print(f"  - PyTorch version: {torch.__version__}")
    
    # GPU memory stats
    total_mem = torch.cuda.get_device_properties(device).total_memory / 1e9
    reserved_mem = torch.cuda.memory_reserved(device) / 1e9
    allocated_mem = torch.cuda.memory_allocated(device) / 1e9
    free_mem = reserved_mem - allocated_mem
    
    print(f"  - Total memory: {total_mem:.2f} GB")
    print(f"  - Reserved memory: {reserved_mem:.2f} GB")
    print(f"  - Allocated memory: {allocated_mem:.2f} GB")
    print(f"  - Free memory in reserved pool: {free_mem:.2f} GB")
else:
    print("❌ GPU not available. Make sure Runtime -> Change runtime type -> GPU is selected")